<a href="https://colab.research.google.com/github/Dkepffl/2022-1-ESAA/blob/main/Assignment/Assignment0415.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# import package
import numpy as np
import pandas as pd

In [6]:
# 데이터 전처리 함수
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

# **CHAPTER 03 평가**
___
- 머신러닝의 과정 : 데이터 가공/변환 - 모델 학습/예측 - 평가(Evaluation)
- 성능 평가 지표(Evaluation Metric)은 일반적으로 모델이 **분류**인지, **회귀**인지에 따라 여러 종류로 나뉜다.
  1. 회귀 : 실제값과 예측값의 오차 평균값(MSE)
  2. 분류 : 실제 결과 데이터와 예측 결과 데이터가 얼마나 정확하고 오류가 적게 발생하는가를 기반으로 평가
- 분류의 성능 평가 지표
  1. 정확도(Accuracy)
  2. 오차행렬(Confusion Matrix)
  3. 정밀도(Precision)
  4. 재현율(Recall)
  5. F1 스코어
  6. ROC AUC
- 분류는 결정 클래스 값의 종류에 따라 긍정/부정과 같은 2개의 결과값만을 가지는 **이진 분류**와 여러 개의 결정 클래스 값을 가지는 **멀티 분류**로 나뉜다.

## **01 정확도**
____

$$정확도 = \frac {예측 결과가 동일한 데이터 건수}{전체 예측 데이터 건수}$$
- 정확도 : 실제 데이터와 예측 데이터가 얼마나 같은지를 판단하는 지표이며 직관적으로 모델 예측 성능을 나타내는 평가 지표이다.
- 이진 분류의 경우, 데이터의 구성에 따라 ML 모델의 성능이 왜곡될 수 있어 정확도 하나만 가지고 성능을 평가하지는 않는다.

In [3]:
from sklearn.base import BaseEstimator

# BaseEstimator 클래스를 상속받아, 아무런 학습을 하지 않고, 성별에 따라 생존자를 예측하는 Classifier
# BaseEstimator 클래스 : Customized 형태의 Estimator를 생성할 수 있다
class MyDummyClassifier(BaseEstimator):
    # fit() 메소드는 아무것도 학습하지 않음
    def fit(self, X , y=None):
        pass
    
    # predict(  메소드는 단순히 Sex feature가 1이면 0 , 그렇지 않으면 1로 예측
    def predict(self, X):
        pred = np.zeros( ( X.shape[0], 1 ))
        for i in range (X.shape[0]) :
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else :
                pred[i] = 1
        
        return pred

In [8]:
# 예제에 필요한 타이타닉 데이터 로드 및 전처리
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

titanic_df = pd.read_csv('/content/drive/MyDrive/2022-1 ESAA/DataSet/titanic_train.csv')

# Feature, Target 데이터셋 분리
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived', axis=1)

# Feature 데이터셋 전처리
X_titanic_df = transform_features(X_titanic_df)

# Train/Test
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, test_size = 0.2, random_state = 0)

In [9]:
# 위에서 생성한 Dummy Classifier를 이용하여 학습/예측/평가 수행. 
myclf = MyDummyClassifier()
myclf.fit(X_train ,y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test , mypredictions)))

Dummy Classifier의 정확도는: 0.7877


- 위의 예제를 보면, 단순한 분류 알고리즘으로 예측을 했으나, 데이터의 구성에 의해 정확도 결과가 약 78.77%로 꽤 높게 나왔다. 따라서 정확도를 평가 지표로 사용할 때는 매우 신중해야 한다.
- Label 값이 불균형하게(imbalanced) 분포할 때, 정확도는 적합한 평가 지표가 아니다. MNIST 데이터셋을 가공하여 불균형한 데이터셋에 대해 정확도를 평가 지표로 사용할 때 어떤 문제가 발생할 수 있는지 살펴보자.

In [14]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score

class MyFakeClassifier(BaseEstimator):
    def fit(self,X,y):
        pass
    
    # 입력값으로 들어오는 X 데이터셋의 크기만큼 모두 0값으로 만들어서 반환
    # 즉, 예측값이 모두 0으로 반환
    def predict(self,X):
        return np.zeros((len(X),1) , dtype = bool)

# 사이킷런의 내장 데이터 셋인 load_digits( )를 이용하여 MNIST 데이터 로딩
digits = load_digits()

# digits 번호(Label 값)가 7이면 True. 7이 아니면 False
# True와 False를 정수형으로 변환
y = (digits.target == 7).astype(int) 

# Train/Test
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state = 11)

In [15]:
# 불균형한 레이블 데이터 분포도 확인
# 0이 1에 비해 매우 많다
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0 과 1의 분포도')
print(pd.Series(y_test).value_counts())

레이블 테스트 세트 크기 : (450,)
테스트 세트 레이블 0 과 1의 분포도
0    405
1     45
dtype: int64


In [16]:
# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train , y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는 : {:.3f}'.format(accuracy_score(y_test ,fakepred)))

모든 예측을 0으로 하여도 정확도는 : 0.900


- 이처럼 불균형한 레이블 데이터셋에서는 정확도로 모델의 성능을 평가할 수 없다.
- 이를 보완하기 위해, 여러 가지 분류 지표와 함께 적용해야 하는데, 이 중 오차 행렬(Confusion Matrix)이 있다.

## **02 오차 행렬**
____
- 오차행렬(Confusion matrix, 혼동행렬) : 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고(confused) 있는지도 함께 보여주는 지표. 즉, 예측 오류 값과 어떤 유형의 예측 오류가 발생하고 있는지를 함께 보여주는 지표. `confusion_matrix()` 메소드를 이용해 계산할 수 있다.

- 오차행렬은 4분면 행렬에서 실제 Label 클래스 값과 예측 Label 클래스 값이 어떠한 유형을 가지고 mapping되는지를 표현한다. 
| |Negative(0)|Positive(1)|
|------|---|---|
|Negative(0)|True Negative|False Positive|
|Positive(1)|False Negative|True Positive|

- mapping 유형
  1. True Negative : 예측값을 Negative(0)으로 예측했고, 실제 값 역시 Negative(0)
  2. False Positive : 예측값을 Positive(1)으로 예측했지만, 실제 값은 Negative(0)
  3. False Negative : 예측값을 Negative(0)으로 예측했지만, 실제 값은 Positive(1)
  4. True Positive : 예측값을 Positive(1)으로 예측했고, 실제 값 역시 Positive(1)


In [18]:
from sklearn.metrics import confusion_matrix

# 앞 예제의 예측 결과인 fakepred와 실제 결과인 y_test의 Confusion Matrix
confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]])

- 오차 행렬

| |Negative(0)|Positive(1)|
|------|---|---|
|Negative(0)|405|0|
|Positive(1)|45|0|

- 오차행렬 결과를 이용해 정확도를 계산할 수 있다.

$$정확도 = \frac {예측 결과가 동일한 데이터 건수}{전체 예측 데이터 건수}  = \frac {(TN + TP)}{(TN + FP + FN + TP}$$ 

- 보통 많은 데이터들은 중점적으로 찾아야 하는 매우 적은 수의 결과값에 Positive를 설정한다. 그런데 불균형한 이진 분류 데이터셋에서는 Positive 데이터 건수가 매우 작아, 이러한 데이터에 기반한 ML 알고리즘은 주로 Positive보다는 Negative로 예측하려는 경향이 더 강해진다. 결과적으로 TN은 매우 커지고, TP는 매우 작아지고, FN이 매우 작으며, FP 역시 매우 작아진다.



## **03 정밀도와 재현율**
____
- 정확도는 분류 모델의 성능을 측정할 수 있는 요소들 중 한 가지일 뿐, 불균형한 데이터셋에서는 성능 지표로서 신뢰가 떨어지는데, 이때 정밀도와 재현율이 평가 지표로 더 선호된다.
- 정밀도와 재현율은 Positive 데이터셋의 예측 성능에 좀 더 초점을 맞춘 지표이다.
1. **정밀도**
  - 예측을 Positive로 한 대상 중에 예측값과 실제값이 Positive로 일치한 데이터의 비율로 양성 예측도라고도 한다.
  - 재현율은 실제 Positive인 데이터를 Negative로 잘못 예측하면 손실이 큰 경우 중요하다.(예 : 암 판단 모델이나 금융 사기 적발 모델)
  - 사이킷런의 `precision_score()` 메서드 사용

$$정밀도 = \frac {TP}{(FP + TP)}$$ 

2. **재현율**
  - 실제 값이 Positive인 대상 중 예측과 실제 값이 Positive로 일치한 데이터의 비율로 민감도(Sensitivity) 혹은 TPR(True Positive Rate)으로 불린다.
  - 정밀도는 실제 Negative인 데이터를 Positive로 잘못 예측하면 손실이 큰 경우 중요하다.(예 : 스팸 메일 여부 판단 모델)
  - 사이킷런의 `recall_score()` 메서드 사용

$$재현율 = \frac {TP}{(FN + TP)}$$ 

- 두 지표는 상호 보완적인 지표이며, 재현율과 정밀도 모두 높은 수치를 얻을 때 모델의 성능이 가장 좋다고 평가한다. 하지만 어느 한 평가 지표만 매우 높고 다른 지표는 매우 낮을 경우는 바람직하지 않다.



In [19]:
# 앞에서 만든 MyFakeClassifier는 TP 값이 하나도 없기 때문에, 정밀도와 재현율 값이 모두 0
from sklearn.metrics import accuracy_score, precision_score , recall_score

print("정밀도:", precision_score(y_test, fakepred))
print("재현율:", recall_score(y_test, fakepred))

정밀도: 0.0
재현율: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
from sklearn.metrics import accuracy_score, precision_score , recall_score , confusion_matrix

# confusion matrix, accuracy, precision, recall 등의 평가 지표를 한 번에 호출하는 함수
def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy , precision ,recall))

In [21]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression

# 원본 데이터를 재로딩, 데이터 가공
titanic_df = pd.read_csv('/content/drive/MyDrive/2022-1 ESAA/DataSet/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived', axis = 1)
X_titanic_df = transform_features(X_titanic_df)

# Train/Test
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.20, random_state=11)

# 타이타닉 생존자를 로지스틱 회귀 모델을 기반으로 예측
lr_clf = LogisticRegression()

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)

# Model Test
get_clf_eval(y_test, pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확도: 0.8492, 정밀도: 0.7742, 재현율: 0.7869


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


- 위의 예제에서 정밀도에 비해 재현율이 낮게 나왔다.
- 재현율 혹은 정밀도를 좀 더 강화할 방법은 무엇일까?

### | 정밀도/재현율 트레이드오프
___